In [11]:
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn import tree

In [12]:
data = pd.read_csv('training_data - training_data.csv')
data

,service_name,service_category
0,"Fade, Beard Trim, Line Up",Barber
1,Crochet (bulk hair),Braids
2,Loc Maintenance w/demi color,Natural Hair
3,Buns & Ponytails,Style
4,"Full Leg Wax, Under arms",Waxing
...,...,...
17995,Fibroids No More Consultation,Special Occasion Style
17996,"Blowout, Root retouch (gray coverage)",Style
17997,"Silk press w/ Trim, Blonding(ADD ON ONLY), Dry...","Natural Hair, Hair Color, Hair Treatments"
17998,"Full Set, No Chip Gel Polish add/service, Nail...",Extensions


In [13]:
train = data[:16000]
train

,service_name,service_category
0,"Fade, Beard Trim, Line Up",Barber
1,Crochet (bulk hair),Braids
2,Loc Maintenance w/demi color,Natural Hair
3,Buns & Ponytails,Style
4,"Full Leg Wax, Under arms",Waxing
...,...,...
15995,"Updo, All Over Color",Hair Color
15996,"Men's Cut, color exprss",Men's Haircut
15997,"Men's Cut Designer, Boy's Cut Designer, Beard ...",Barber
15998,"Shampoo, Flat Iron",Style


In [14]:
validate = data[16000:]
validate

,service_name,service_category
16000,"Gel Manicure, Pedicure, Add Glitter","Manicure, Pedicure"
16001,Acrylic FILL,Manicure
16002,"Crochet Braids, Demi color",Braids
16003,"Brazilian Blowout, Full Color, Extensions","Straightening Treatments, Extensions"
16004,"Color correction, Adding olaplex to color serv...",Hair Color
...,...,...
17995,Fibroids No More Consultation,Special Occasion Style
17996,"Blowout, Root retouch (gray coverage)",Style
17997,"Silk press w/ Trim, Blonding(ADD ON ONLY), Dry...","Natural Hair, Hair Color, Hair Treatments"
17998,"Full Set, No Chip Gel Polish add/service, Nail...",Extensions


In [15]:
test = pd.read_csv('testing_data - testing_data.csv')
test

,service_name
0,Microlink silk press
1,"Full sewin no leaveout, Eyebrow Arch + enhance..."
2,Corrective Color no cut
3,KAE Classic *$70 SPECIAL*
4,loc style
...,...
1995,"Brow Tint, Brow Clean Up"
1996,"Relaxer, Demi-permanent hair color, Shine Enha..."
1997,Back 2 School Jumbo Box Braid Special
1998,Mens Twist Sponge/Specialty


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train.service_name)
X_train_counts.shape

(16000, 3468)

In [17]:
X_train_counts

<16000x3468 sparse matrix of type '<class 'numpy.int64'>'
	with 81768 stored elements in Compressed Sparse Row format>

In [20]:
X_train_counts.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(16000, 3468)

In [9]:
X_train_tf

<16000x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 81768 stored elements in Compressed Sparse Row format>

In [25]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(16000, 3468)

In [28]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, train.service_category)

In [29]:
#Use cross validating to avoid overfitting, we can repeat this process for 10 times and use average as measure
v_new_counts = count_vect.transform(validate.service_name)
v_new_tfidf = tfidf_transformer.transform(v_new_counts)
v_predicted = clf.predict(v_new_tfidf)
v_predicted

array(['Manicure', 'Manicure', 'Braids', ..., 'Natural Hair', 'Manicure',
       'Braids'], dtype='<U133')

In [30]:
X_new_counts = count_vect.transform(test.service_name)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [31]:
predicted = clf.predict(X_new_tfidf)
predicted

array(['Natural Hair', 'Weaves', "Women's Haircut", ..., 'Braids',
       "Men's Haircut", "Women's Haircut"], dtype='<U133')

In [32]:
predicted.shape

(2000,)

In [36]:
pd.DataFrame(predicted).to_csv("Siyu_Xiao_submission_problem_1.csv", header=['service_category'])